In [1]:
%pip install --quiet --upgrade -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Dependencies and Tools

In [2]:
from langchain_core.messages import HumanMessage

In [3]:
# Define here maximum web search results for the web scraper

MAX_WEB_SEARCH_RESULTS = 5

In [4]:
# Agent tools

tools = []

In [5]:
import os

os.environ["LANGSMITH_TRACING"] = "true"

In [6]:
#Env File

from dotenv import load_dotenv

load_dotenv("trustme.env");

In [7]:
from langchain.prompts import SystemMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    '''
    Quando l'utente chiede di valutare la fiducia o l'affidabilità di una persona specifica 
    (es. 'Quanto posso fidarmi di Mario Rossi?'), usa il tool 'search' passando il nome della persona.
    '''
)

In [8]:
# LLM 

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

In [9]:
#Embeddings Model

from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDINGS_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDINGS_API_VERSION"),
)

In [10]:
# Vector DB

from langchain_community.vectorstores.azuresearch import AzureSearch

vector_store = AzureSearch(
    azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    azure_search_key=os.getenv("AZURE_SEARCH_API_KEY"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    embedding_function=embeddings.embed_query,
)

In [11]:
# Checkpointer on PostgreSQL
from langgraph.checkpoint.postgres import PostgresSaver
import psycopg

conn = psycopg.connect(os.getenv("POSTGRES_CONNECTION_STRING"))

if conn is not None:
    checkpointer = PostgresSaver(conn)
else:
    raise ValueError("Non sono riuscito ad inizializzare PostgreSQL")

In [12]:
# Web Search Tool

from langchain_tavily import TavilySearch

search = TavilySearch(
    max_results=MAX_WEB_SEARCH_RESULTS,
    description="Usa questo tool per cercare informazioni su una persona specifica quando l'utente chiede di valutare la fiducia o l'affidabilità di qualcuno, fornendo il nome."
)

tools.append(search)

### Implementation

In [13]:
from langgraph.prebuilt import create_react_agent

personal_agent = create_react_agent(llm, tools, checkpointer=checkpointer)

In [14]:
import random

thread_id = "thread-" + ''.join([str(random.randint(0,9)) for i in range(10)])

In [17]:
while True:

    user_message = input("Query (exit per uscire)")
    # SAMPLE: Valuta la fiducia di Mario Rossi.

    if user_message == "exit" or user_message == "esci" or not user_message:
        break

    #TO-DO: prompting for user message

    response = personal_agent.invoke(
        {
            "messages": [HumanMessage(content=user_message)]
        },
        config={
            "thread_id": thread_id 
        }
    )

    for message in response['messages']:
        print(message.content)
    
    if "tool_calls" in response and len(response["tool_calls"]) > 0:

        for tool_call in response["tool_calls"]:
            print(f"Chiamata metodo {tool_call['name']}")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block